In [10]:
# Cell 1 - Imports + paths
import numpy as np
import pandas as pd

IN_CSV  = r"C:\Users\aysen\OneDrive\Masaüstü\data mining proje factorio\merged_factorio.csv"
OUT_CSV = r"C:\Users\aysen\OneDrive\Masaüstü\data mining proje factorio\analysis_ready.csv"

In [11]:
# Cell 2 - Load + dedup by tick (keep last)
df = pd.read_csv(IN_CSV)
df = df.sort_values("tick").groupby("tick", as_index=False).last().copy()

print("rows:", len(df), "cols:", df.shape[1])
print("tick range:", df["tick"].min(), "->", df["tick"].max())
df.head(3)

rows: 710 cols: 477
tick range: 1026000 -> 3578400


C:\Users\aysen\AppData\Local\Temp\ipykernel_28628\3461218500.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = df.sort_values("tick").groupby("tick", as_index=False).last().copy()


,tick,time_s,pollution,buildings__boiler,buildings__offshore-pump,buildings__assembling-machine-1,buildings__burner-mining-drill,buildings__stone-furnace,buildings__electric-mining-drill,buildings__lab,production__transport-belt,production__inserter,production__pipe,production__stone-furnace,production__wood,production__coal,production__stone,production__iron-ore,production__copper-ore,production__iron-plate,production__copper-plate,production__iron-gear-wheel,production__copper-cable,production__electronic-circuit,production__automation-science-pack,consumption__iron-chest,consumption__transport-belt,consumption__underground-belt,consumption__splitter,consumption__burner-inserter,consumption__inserter,consumption__long-handed-inserter,consumption__fast-inserter,consumption__small-electric-pole,consumption__pipe,consumption__stone-brick,consumption__boiler,consumption__steam-engine,consumption__burner-mining-drill,consumption__electric-mining-drill,consumption__offshore-pump,consumption__stone-furnace,consumption__assembling-machine-1,consumption__lab,consumption__coal,consumption__stone,consumption__iron-ore,consumption__copper-ore,consumption__iron-plate,consumption__copper-plate,consumption__iron-gear-wheel,consumption__copper-cable,consumption__electronic-circuit,consumption__automation-science-pack,consumption__science,fluids__water,fluids__steam,power__satisfaction,power__capacity,power__production,power__demand,consumption__pipe-to-ground,consumption__logistic-science-pack,production__logistic-science-pack,consumption__steel-plate,consumption__wooden-chest,production__stone-brick,production__steel-plate,consumption__steel-furnace,buildings__steel-furnace,consumption__radar,production__assembling-machine-1,consumption__steel-chest,consumption__assembling-machine-2,buildings__assembling-machine-2,consumption__solar-panel,production__splitter,consumption__fast-splitter,consumption__iron-stick,production__iron-stick,consumption__medium-electric-pole,consumption__big-electric-pole,consumption__decider-combinator,consumption__constant-combinator,consumption__small-lamp,consumption__rail,consumption__engine-unit,buildings__solar-panel,production__engine-unit,consumption__car,consumption__locomotive,consumption__rail-chain-signal,consumption__train-stop,consumption__rail-signal,consumption__fast-transport-belt,production__underground-belt,consumption__fast-underground-belt,consumption__cargo-wagon,consumption__display-panel,consumption__pumpjack,...,M100_count,M101_state,M101_item,M101_count,M102_state,M102_item,M102_count,M103_state,M103_item,M103_count,M104_state,M104_item,M104_count,M105_state,M105_item,M105_count,M106_state,M106_item,M106_count,M107_state,M107_item,M107_count,M108_state,M108_item,M108_count,M109_state,M109_item,M109_count,M110_state,M110_item,M110_count,M111_state,M111_item,M111_count,M112_state,M112_item,M112_count,M113_state,M113_item,M113_count,M114_state,M114_item,M114_count,M115_state,M115_item,M115_count,M116_state,M116_item,M116_count,M117_state,M117_item,M117_count,M118_state,M118_item,M118_count,M119_state,M119_item,M119_count,M120_state,M120_item,M120_count,M121_state,M121_item,M121_count,M122_state,M122_item,M122_count,M123_state,M123_item,M123_count,M124_state,M124_item,M124_count,M125_state,M125_item,M125_count,M126_state,M126_item,M126_count,M127_state,M127_item,M127_count,M128_state,M128_item,M128_count,M129_state,M129_item,M129_count,M130_state,M130_item,M130_count,M131_state,M131_item,M131_count,M132_state,M132_item,M132_count,M133_state,M133_item,M133_count
0,1026000,17100,2112.483483,1,1,18,18.0,104,17,2,56,17,92,29,91,5977,699,18793,7308,12792,2261,2751,3954,1183,209,11,2236,8,7,1,992,3,14,180,109,22,6,13,23,18,1,131,18,2,11328,2770,19978,8696,18751,7265,4289,4000,1257,261,209,23740.843320,226168.827625,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,None,NaN,N

In [12]:
# Cell 3 - Ensure numeric time + power
df["tick"] = pd.to_numeric(df["tick"], errors="coerce").astype("int64")
df["logsim_power"] = pd.to_numeric(df["logsim_power"], errors="coerce")

# time axis: prefer time_s if exists, else derive from tick (Factorio: 60 ticks = 1 second)
if "time_s" in df.columns:
    df["time_s"] = pd.to_numeric(df["time_s"], errors="coerce")
else:
    df["time_s"] = df["tick"] / 60.0

# delta minutes
df["dt_min"] = df["time_s"].diff() / 60.0
df.loc[df["dt_min"] <= 0, "dt_min"] = np.nan

df[["tick","time_s","dt_min","logsim_power"]].head(10)

,tick,time_s,dt_min,logsim_power
0,1026000,17100,NaN,NaN
1,1029600,17160,1.0,NaN
2,1033200,17220,1.0,NaN
3,1036800,17280,1.0,NaN
4,1040400,17340,1.0,NaN
5,1044000,17400,1.0,NaN
6,1047600,17460,1.0,NaN
7,1051200,17520,1.0,NaN
8,1054800,17580,1.0,NaN
9,1058400,17640,1.0,NaN


In [13]:
# Cell 4 - Build per_min production rates from cumulative production__* columns
prod_cum_cols = [c for c in df.columns if c.startswith("production__") and not c.endswith("_per_min")]
df[prod_cum_cols] = df[prod_cum_cols].apply(pd.to_numeric, errors="coerce")

rate_cols = []
for c in prod_cum_cols:
    rc = c + "_per_min"
    df[rc] = df[c].diff() / df["dt_min"]
    rate_cols.append(rc)

# clean rates
df[rate_cols] = df[rate_cols].replace([np.inf, -np.inf], np.nan)
df[rate_cols] = df[rate_cols].clip(lower=0)

print("cumulative production cols:", len(prod_cum_cols))
print("rate cols:", len(rate_cols))
df[["tick"] + rate_cols[:8]].head(10)

cumulative production cols: 24
rate cols: 24


,tick,production__transport-belt_per_min,production__inserter_per_min,production__pipe_per_min,production__stone-furnace_per_min,production__wood_per_min,production__coal_per_min,production__stone_per_min,production__iron-ore_per_min
0,1026000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1029600,0.0,0.0,0.0,0.0,0.0,38.0,0.0,152.0
2,1033200,0.0,0.0,0.0,0.0,0.0,41.0,0.0,150.0
3,1036800,0.0,0.0,0.0,0.0,0.0,37.0,0.0,147.0
4,1040400,0.0,0.0,0.0,0.0,0.0,35.0,0.0,151.0
5,1044000,0.0,0.0,0.0,0.0,0.0,36.0,0.0,152.0
6,1047600,0.0,0.0,0.0,0.0,0.0,36.0,0.0,150.0
7,1051200,0.0,0.0,0.0,0.0,0.0,29.0,0.0,147.0
8,1054800,0.0,0.0,0.0,0.0,0.0,30.0,0.0,151.0
9,1058400,0.0,0.0,0.0,0.0,0.0,27.0,0.0,152.0


In [16]:
# Cell 4b - (After you create rate_cols and throughput_total_per_min)
# Replace NaNs in rates with 0 (means no production observed in that minute)
df[rate_cols] = df[rate_cols].fillna(0)

# throughput recompute (safe)
df["throughput_total_per_min"] = df[rate_cols].sum(axis=1)

print("throughput>0:", (df["throughput_total_per_min"] > 0).sum())
print("logsim_power non-null:", df["logsim_power"].notna().sum())
print("logsim_power > 0:", (df["logsim_power"].fillna(0) > 0).sum())

throughput>0: 709
logsim_power non-null: 365
logsim_power > 0: 365


In [14]:
# Cell 5 - Throughput & active flag
df["throughput_total_per_min"] = df[rate_cols].sum(axis=1, skipna=True)

df["is_active"] = (df["logsim_power"].fillna(0) > 0) & (df["throughput_total_per_min"].fillna(0) > 0)

print("power>0:", (df["logsim_power"].fillna(0) > 0).sum())
print("throughput>0:", (df["throughput_total_per_min"].fillna(0) > 0).sum())
print("active:", df["is_active"].sum())

df.loc[df["is_active"], ["tick","logsim_power","throughput_total_per_min"]].head(10)

power>0: 365
throughput>0: 709
active: 365


,tick,logsim_power,throughput_total_per_min
345,2268000,90000.0,2755.0
346,2271600,90000.0,2415.0
347,2275200,90000.0,2094.0
348,2278800,86776.0,2482.0
349,2282400,84867.0,2425.0
350,2286000,84841.0,2297.0
351,2289600,88596.0,2183.0
352,2293200,84050.0,2222.0
353,2296800,87647.0,2129.0
354,2300400,89925.0,2272.0


In [17]:
# Cell 5b - Active window
df["is_active"] = (df["logsim_power"].fillna(0) > 0) & (df["throughput_total_per_min"] > 0)

dfA = df[df["is_active"]].copy()

print("Active rows:", len(dfA))
print("Active tick range:", dfA["tick"].min(), "->", dfA["tick"].max())

# show first active rows to verify
dfA[["tick","time_s","logsim_power","throughput_total_per_min"]].head(10)

Active rows: 365
Active tick range: 2268000 -> 3578400


,tick,time_s,logsim_power,throughput_total_per_min
345,2268000,37800,90000.0,2755.0
346,2271600,37860,90000.0,2415.0
347,2275200,37920,90000.0,2094.0
348,2278800,37980,86776.0,2482.0
349,2282400,38040,84867.0,2425.0
350,2286000,38100,84841.0,2297.0
351,2289600,38160,88596.0,2183.0
352,2293200,38220,84050.0,2222.0
353,2296800,38280,87647.0,2129.0
354,2300400,38340,89925.0,2272.0


In [15]:
# Cell 6 - Keep only what we need (smaller file, faster later)
keep_cols = (
    ["tick","time_s","dt_min","pollution","logsim_power",
     "throughput_total_per_min","is_active"]
    + prod_cum_cols
    + rate_cols
)

# keep only columns that exist (pollution might be absent sometimes)
keep_cols = [c for c in keep_cols if c in df.columns]

df_out = df[keep_cols].copy()

print("analysis_ready shape:", df_out.shape)
df_out.head(3)

analysis_ready shape: (710, 55)


,tick,time_s,dt_min,pollution,logsim_power,throughput_total_per_min,is_active,production__transport-belt,production__inserter,production__pipe,production__stone-furnace,production__wood,production__coal,production__stone,production__iron-ore,production__copper-ore,production__iron-plate,production__copper-plate,production__iron-gear-wheel,production__copper-cable,production__electronic-circuit,production__automation-science-pack,production__logistic-science-pack,production__stone-brick,production__steel-plate,production__assembling-machine-1,production__splitter,production__iron-stick,production__engine-unit,production__underground-belt,production__storage-tank,production__transport-belt_per_min,production__inserter_per_min,production__pipe_per_min,production__stone-furnace_per_min,production__wood_per_min,production__coal_per_min,production__stone_per_min,production__iron-ore_per_min,production__copper-ore_per_min,production__iron-plate_per_min,production__copper-plate_per_min,production__iron-gear-wheel_per_min,production__copper-cable_per_min,production__electronic-circuit_per_min,production__automation-science-pack_per_min,production__logistic-science-pack_per_min,production__stone-brick_per_min,production__steel-plate_per_min,production__assembling-machine-1_per_min,production__splitter_per_min,production__iron-stick_per_min,production__engine-unit_per_min,production__underground-belt_per_min,production__storage-tank_per_min
0,1026000,17100,NaN,2112.483483,NaN,0.0,False,56,17,92,29,91,5977,699,18793,7308,12792,2261,2751,3954,1183,209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1029600,17160,1.0,2125.544060,NaN,538.0,False,56,17,92,29,91,6015,699,18945,7356,12942,2291,2794,4011,1203,209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,38.0,0.0,152.0,48.0,150.0,30.0,43.0,57.0,20.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1033200,17220,1.0,2136.843736,NaN,523.0,False,56,17,92,29,91,6056,699,19095,7394,13091,2315,2837,4071,1221,209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,41.0,0.0,150.0,38.0,149.0,24.0,43.0,60.0,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df["logsim_power_plot"] = df["logsim_power"].fillna(0)

In [19]:
# Cell 7 - Save
df_out.to_csv(OUT_CSV, index=False)
print("Saved:", OUT_CSV)

Saved: C:\Users\aysen\OneDrive\Masaüstü\data mining proje factorio\analysis_ready.csv
